#### 一. volatile
1. 线程共享变量模型   
 1. Java中, 每个线程都有自己独立的执行内存空间.   
 2. 当线程需要存取一个共享变量时, 要先获取锁定, 并从主内存拷贝共享变量到自己的线程执行空间里  
   当线程操作完这个变量后, 释放锁并将结果写会主内存  
2. 线程安全问题  
 假设一个线程从共享内存中拷贝了共享变量后释放了锁, 而后线程内部操作该变量. 此时, 有另一个线程又从主内存拷贝共享变量, 这个新线程拷贝的结果会是上一个线程没做出修改之前的值, 从而产生安全问题  
3. `volatile`关键字  
 volatile关键字声明的变量可以做到如下保证 :  
  1. 其他线程对共享变量的修改, 可以即时反应在当前线程中  
  2. 确保当前线程对volatile变量的修改, 能即时写回到 共享主内存中, 并被其他线程看见  
  3. volatile声明的变量, 将破事所有线程均读写主内存中的变量值, 从而使得volatile变量在多线程间可见    
4. -server模式  
 java在'-server'模式下, JVM不回去读取未发生变化且未标记为volatile的共享变量, 使得变量的修改在线程之间是不可见的

#### 二. Synchronized
1. Synchronized使用方法  
 1. 关键字在方法上: 
   ```java
   public synchronized void method(){}
   ```
 2. 关键字在方法内的同步代码块
   ```java
   public synchronized void method(Object obj){
       ...
       synchronzied(obj){ }
       ...
    }
   ```
 3. 关键字在静态方法上   
  关键字在静态方法上, 相当于锁加到当前Class上, 所有对该方法的调用, 都要现货的该Class的锁
  ```java
    public synchronized static void method(Object obj){}
  ```
2. synchronized与wait,notify合用  
 单一的synchronized虽然可以保证线程安全, 但需要配合其它线程方法, 才能表示复杂逻辑的线程交互  
 1. `obj.wait()`  
    1. 使用方法  
     ```java
         synchronized(obj){
            while(条件){
                obj.wait();
                // 收到通知后, 继续执行
            }
        }
     ```
    2. 使用wait()之前, 需要获取对象锁.   
    3. 其次, wait()方法要写在while循环中, 并指明跳出循环的条件 :  
        因为wait()别唤醒后, 原先的判断条件可能已经发生改变, 需要再次判断  
    4. 最后, wait()方法执行时, 线程会释放得到的obj独占锁, 并进入'等待阻塞'状态, 等待其它线程执行该`obj`锁的`notify()` 
 2. `obj.notify()`  
   当等待在obj上的线程收到一个`obj.notify()`时, 就能重新获得obj的锁.值得注意的是以下3点 :  
    1. 当线程执行完`obj.notify()`后, 不会立刻释放锁, 而是等待synchronized代码块中的代码全部执行完毕后再释放锁  
    2. 如果有多个线程在方法`obj.wait()`中, 则只会随机选择一个线程唤醒  
    3. `obj.notifyAll()`会唤醒所有在执行`obj.wait()`的线程

#### 三. 重入锁ReentrantLock
1. 重入锁分为公平锁和非公平锁, 通过如下方式指定   
```java
public ReentrantLock(boolean fair)
```
  1. 公平锁保证在锁等待队列中, 各线程是公平的, 因此不会存在插队情况, 对锁的获取总是先进先出  
  2. 非公平锁没有这个保证, 后申请锁的线程可能会先获得锁
2. `lock()` :   等待获得锁, 等待过程中不会响应中断异常, 等到执行unlock释放锁时, 再响应中断
3. `lockinterruptibly()` : 与lock的唯一区别在于,等待过程中会响应中断,抛出中断异常  
4. `tryLock()` :  
 tryLock尝试在指定时间内获取锁, 并在等待时间内可以响应中断异常.  
 获取成功返回true, 否则返回false

In [8]:
import java.util.concurrent.TimeUnit;
import java.util.concurrent.locks.ReentrantLock;
public class LockTest{
    public static Runnable createTask(ReentrantLock lock){
        return new Runnable() {
            @Override
            public void run() {
                while (true){
                    try{
                        // tryLock在指定时间内等待锁, 并在等待时间内可以响应中断异常
                        if (lock.tryLock(500, TimeUnit.MILLISECONDS)){
                            try {
                                System.out.println("locked "+Thread.currentThread().getName());
                                Thread.sleep(2000);
                            }finally {
                                lock.unlock();
                                System.out.println("unlocked "+Thread.currentThread().getName());
                            }
                            break;
                        }else {
                            System.out.println("unable to lock "+Thread.currentThread().getName());
                        }
                    }catch (InterruptedException e){
                        System.out.println(Thread.currentThread().getName()+" is Interrupted");
                    }
                }
            }import java.util.concurrent.locks.Condition;
import java.util.concurrent.locks.ReentrantLock;
//声明锁
final ReentrantLock loc = new ReentrantLock();
final Condition notEmpty = lock.newCondition();
final Condition notFull =  lock.newCondition();
///////////////////////  put操作  ///////////////////
public void put(E e) throws InterruptedException {
    final ReentrantLock lock = this.lock;
    lock.lockInterruptibly();  // 竞争锁
    try {
        while (count == items.length)  // 如果队列已满, 释放锁进入等待, 等待队列有剩余空间
            notFull.await();
        enqueue(e);
    } finally {
        lock.unlock(); // 释放lock
    }
}
private void enqueue(E e) {
    // assert lock.isHeldByCurrentThread();
    // assert lock.getHoldCount() == 1;
    // assert items[putIndex] == null;
    final Object[] items = this.items;
    items[putIndex] = e;
    if (++putIndex == items.length) putIndex = 0;
    count++;
    notEmpty.signal(); // 通知take线程, 队列已有数据
}

////////////////////////  take操作 ///////////////////////
public E take() throws InterruptedException {
    final ReentrantLock lock = this.lock;
    lock.lockInterruptibly();  //对take做同步
    try {
        while (count == 0) // 若队列已空, 等待一个非空condition
            notEmpty.await();
        return dequeue();
    } finally {
        lock.unlock(); // 释放锁
    }
}
private E dequeue() {
    // assert lock.isHeldByCurrentThread();
    // assert lock.getHoldCount() == 1;
    // assert items[takeIndex] != null;
    final Object[] items = this.items;
    @SuppressWarnings("unchecked")
    E e = (E) items[takeIndex];
    items[takeIndex] = null;
    if (++takeIndex == items.length) takeIndex = 0;
    count--;
    if (itrs != null)
        itrs.elementDequeued();
    notFull.signal();  // 通知put线程已有空闲空间
    return e;
}
        };
    }
    public static Runnable createTask2(ReentrantLock lock){
        return new Runnable() {
            @Override
            public void run() {
                while (true){
                    try{
                        lock.lock();  // 等待锁时不会响应中断异常, 等到执行unlock释放锁时, 再响应中断
                        try {
                            System.out.println("locked "+Thread.currentThread().getName());
                            Thread.sleep(2000);
                        }finally {
                            System.out.println("unlocked "+Thread.currentThread().getName());
                            lock.unlock(); // 释放锁后, 响应中断, 产生中断异常
                        }
                        break;
                    }catch (InterruptedException e){  // 响应中断, 打印"is Interrupted"后继续执行
                        System.out.println(Thread.currentThread().getName()+" is Interrupted");
                    }
                }
            }
        };
    }
    public static Runnable createTask3(ReentrantLock lock){
        return new Runnable() {
            @Override
            public void run() {
                while (true){
                    try{
                        lock.lockInterruptibly();  // 等待锁过程中可以响应中断异常响应中断
                        try {
                            System.out.println("locked "+Thread.currentThread().getName());
                            Thread.sleep(2000);
                        }finally {
                            System.out.println("unlocked "+Thread.currentThread().getName());
                            lock.unlock(); // 释放锁后, 响应中断, 产生中断异常
                        }
                        break;
                    }catch (InterruptedException e){  // 响应中断, 打印"is Interrupted"后继续执行
                        System.out.println(Thread.currentThread().getName()+" is Interrupted");
                    }
                }
            }
        };
    }
     public static void main(String[] args) throws InterruptedException {
        ReentrantLock lock = new ReentrantLock();
        Thread firstThread = new Thread(createTask3(lock), "first thread");
        Thread secondThread = new Thread(createTask3(lock), "second thread");
        firstThread.start();
        secondThread.start();
        Thread.sleep(600);
        // 主函数试图中断第二个线程
        secondThread.interrupt();
    }
}

#### 四. 读写锁ReadWriteLock
1. ReadWriteLock读写分离锁, 可以有效提升系统性能  
 '读读'之间并未发生相互等待, 但是'读写','写写'之间仍会发锁竞争  
2. 因此, ReadWriteLock读写分离锁, 允许多线程之间同时执行读操作, 而写写操作和读写操作之间仍然需要互相等待和持有锁

In [10]:
// 声明读写分离锁
import java.util.concurrent.locks.ReentrantReadWriteLock;

ReentrantReadWriteLock lock = new ReentrantReadWriteLock();
ReentrantReadWriteLock.ReadLock readLock = lock.readLock();
ReentrantReadWriteLock.WriteLock writeLock = lock.writeLock();

In [ ]:
// 读写分离锁的使用
public void handleRead(ReentrantReadWriteLock.ReadLock readLock) throws InterruptedException {
    try{
        readLock.lock();
        Thread.sleep(500);  // 模拟读操作
    }finally {
        readLock.unlock();
    }

}
public void handleWrite(ReentrantReadWriteLock.WriteLock writeLock) throws InterruptedException {
    try{
        writeLock.lock();
        Thread.sleep(8000);  // 写操作费时
    }finally {
        writeLock.unlock();
    }

}

#### 五. Condition对象
1. Condition.await()与Lock的关系, 就像obj.wait()与synchronized的关系     
 因此, 内部锁Synchronized与重入锁ReentrantLock在功能上有重复,而且ReentrantLock需要在finally代码块中显式的调用`lock.unlock()`释放锁,   
 而内部锁会自动释放锁, 因此往往内部锁+obj.wait()的使用场景更多, 但重入锁+Condition的组合效率更高
2. 实例化  
```java
lock.newCondition()
```
3. 调用  
```java
  void await():释放锁, 线程进入等待, 等待其它线程使用signal()或者sigmalAll()方法时, 该线程重新获得锁并继续执行.   
               等待时可以响应中断  
  void awaitUninterruptibly() : 与await()相似, 但是等待时, 不响应中断  
  void signal(): 唤醒一个等待Condition锁的线程  
```
4. 如下, 以ArrayBlockingQueue为例, 展示Condition锁在它的put与take方法中的实现:  

In [ ]:
import java.util.concurrent.locks.Condition;
import java.util.concurrent.locks.ReentrantLock;
//声明锁
final ReentrantLock loc = new ReentrantLock();
final Condition notEmpty = lock.newCondition();
final Condition notFull =  lock.newCondition();
///////////////////////  put操作  ///////////////////
public void put(E e) throws InterruptedException {
    final ReentrantLock lock = this.lock;
    lock.lockInterruptibly();  // 竞争锁
    try {
        while (count == items.length)  // 如果队列已满, 释放锁进入等待, 等待队列有剩余空间
            notFull.await();
        enqueue(e);
    } finally {
        lock.unlock(); // 释放lock
    }
}
private void enqueue(E e) {
    // assert lock.isHeldByCurrentThread();
    // assert lock.getHoldCount() == 1;
    // assert items[putIndex] == null;
    final Object[] items = this.items;
    items[putIndex] = e;
    if (++putIndex == items.length) putIndex = 0;
    count++;
    notEmpty.signal(); // 通知take线程, 队列已有数据
}

////////////////////////  take操作 ///////////////////////
public E take() throws InterruptedException {
    final ReentrantLock lock = this.lock;
    lock.lockInterruptibly();  //对take做同步
    try {
        while (count == 0) // 若队列已空, 等待一个非空condition
            notEmpty.await();
        return dequeue();
    } finally {
        lock.unlock(); // 释放锁
    }
}
private E dequeue() {
    // assert lock.isHeldByCurrentThread();
    // assert lock.getHoldCount() == 1;
    // assert items[takeIndex] != null;
    final Object[] items = this.items;
    @SuppressWarnings("unchecked")
    E e = (E) items[takeIndex];
    items[takeIndex] = null;
    if (++takeIndex == items.length) takeIndex = 0;
    count--;
    if (itrs != null)
        itrs.elementDequeued();
    notFull.signal();  // 通知put线程已有空闲空间
    return e;
}

#### 六. Semphore信号量
1. 无论是内部锁synchronized, 还是重入锁ReentrantLock, 一次都只允许一个线程访问一个资源, 而信号量允许多个线程同时访问某一个资源  
2. 构造函数如下:  
```java
public Semaphore(int permits)
public Semaphore(int permits,boolean fair) // 第二个参数制定是否公平
```
3. 主要方法  
  1. acquire() : 尝试获得一个准入许可, 若无法获得则进入等待, 等待期间响应中断  
  2. acquireUninterruptibly() : 与acquire()类似, 但是等待期间不响应中断  
  3. tryAcquire() : 尝试获取许可, 立即返回. 成功则返回true, 失败则返回false  
  4. tryAcquire(long timeout, TimeUnit unit);  
  5. release() : 线程访问完资源后, 释放一个许可, 以使其它等待许可的线程可以进行资源访问  

4. 如下例子, 是一个含有100个资源的资源池, 控制同时只能有100个线程访问该池中的资源

In [6]:
import java.util.concurrent.Semaphore;
class Pool {
    private static final int MAX_AVAILABLE = 100;
    // 最大100个许可
    private final Semaphore available = new Semaphore(MAX_AVAILABLE, true);

    public Object getItem() throws InterruptedException {
        available.acquire(); // 申请一个许可, 最多有100个线程同时使用getItem()这个方法
        return getNextAvailableItem();
    }

    public void putItem(Object x) {
        if (markAsUnused(x))
            available.release();
    }

    // Not a particularly efficient data structure; just for demo
    // items包含100个可使用的资源,used数组记录该资源是否已被使用
    protected Object[] items = new Object[MAX_AVAILABLE];
    protected boolean[] used = new boolean[MAX_AVAILABLE];

    protected synchronized Object getNextAvailableItem() {
        for (int i = 0; i < MAX_AVAILABLE; ++i) {
            if (!used[i]) {
                used[i] = true;
                return items[i];  // 从items中找到一个未被使用的资源, 然后返回
            }
        }
        return null; // not reached
    }

    protected synchronized boolean markAsUnused(Object item) {
        for (int i = 0; i < MAX_AVAILABLE; ++i) {
            if (item == items[i]) { // 找到给定项的索引
                if (used[i]) {
                    used[i] = false; // 将给定项设置为未被使用
                    return true;
                } else
                    return false;
            }
        }
        return false;
    }

}

#### 七.ThreadLocal
1. ThreadLocal完全不提供锁,而是以空间换取时间, 为每个线程提供独立的变量, 以保证线程安全.  
2. `ThreadLocal.set(obj)`的对象, 必须是在各自线程内部创建的独立实例

#### 八. 减小锁力度与锁分离

#### 九. 无锁并行计算

#### 十. CountDownLatch与CyclicBarrier

#### 十一. 并发数据结构